In [3]:
import glob
import pandas as pd
import shutil
import os
from xml.etree import ElementTree as et
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from helper_function import *

In [4]:
ExtractLabel('test.xml')

[['chlorotic-and-drying-of-leaves_jpg.rf.ded356f6a70cb5b5cf7c7518664bec05.jpg',
  '640',
  '640',
  'Early_blight',
  '239',
  '433',
  '176',
  '416']]

In [8]:
path = 'G:\\Graduation Project\\AI\\vision\\Data Collection & PreProcessing\\Tomato Leaf Disease.v63i.voc\\train\\'

In [9]:
# Extarct Images & Label Paths
img_paths, lbl_paths = getPaths(path)
   
# Extraxt Annotaions
missing_counter = 0
all_annotations = []
for lbl_path in tqdm(lbl_paths):
    filepath = r'{}'.format(lbl_path)
    try :
        all_annotations.append(ExtractLabel(filepath))
    except FileNotFoundError:
        missing_counter += 1
print('Number Of Missing Image {}'.format(missing_counter))


all_annotations = [x for xs in tqdm(all_annotations) for x in xs]
# Convert To DataFrame
all_annotationsdf = pd.DataFrame(all_annotations, columns = ['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax'])

# Convert Numerical Columns from object to int
cols = ['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax']
all_annotationsdf[cols] = all_annotationsdf[cols].astype(float)

100%|██████████| 9039/9039 [00:30<00:00, 298.05it/s]


Number Of Missing Image 6


100%|██████████| 9033/9033 [00:00<00:00, 365804.93it/s]


In [10]:
all_annotationsdf

,filename,width,height,name,xmin,xmax,ymin,ymax
0,02cb8c4e-4542-4020-bb46-025de54d97ae___PSU_CG-...,512.0,512.0,Mosaic Virus,53.0,455.0,136.0,406.0
1,02cb8c4e-4542-4020-bb46-025de54d97ae___PSU_CG-...,512.0,512.0,Mosaic Virus,60.0,462.0,109.0,379.0
2,02cb8c4e-4542-4020-bb46-025de54d97ae___PSU_CG-...,512.0,512.0,Mosaic Virus,136.0,405.0,60.0,461.0
3,03b6acc7-41a6-4670-b47c-bef6d4baea18___PSU_CG-...,512.0,512.0,Mosaic Virus,143.0,403.0,44.0,386.0
4,03b6acc7-41a6-4670-b47c-bef6d4baea18___PSU_CG-...,512.0,512.0,Mosaic Virus,143.0,403.0,44.0,386.0
...,...,...,...,...,...,...,...,...
33846,zuRDp_jpg.rf.f698767acb746d94164647df24f3e25c.jpg,512.0,512.0,Leaf Miner,211.0,335.0,193.0,306.0
33847,zuRDp_jpg.rf.f698767acb746d94164647df24f3e25c.jpg,512.0,512.0,Leaf Miner,286.0,337.0,83.0,147.0
33848,zuRDp_jpg.rf.f698767acb746d94164647df24f3e25c.jpg,512.0,512.0,Leaf Miner,359.0,392.0,112.0,161.0
33849,zuRDp_jpg.rf.f698767acb746d94164647df24f3e25c.jpg,512.0,512.0,Leaf Miner,390.0,459.0,66.0,172.0


In [24]:
class_diseas = all_annotationsdf['name'].unique()
class_diseas

array(['Mosaic Virus', 'Early Blight', 'Late Blight', 'Septoria',
       'Yellow Leaf Curl Virus', 'Healthy', 'Leaf Mold', 'Leaf Miner',
       'Spider Mites'], dtype=object)

In [14]:
os.makedirs(r'Data\Train')
os.mkdir(r'Data\Val')
os.mkdir(r'Data\Test')

In [49]:
def creat_folder(class_names, path):
    for class_name in class_names:
        os.mkdir('{}\\{}'.format(path, class_name))

In [20]:
for class_name in all_annotationsdf['name'].unique():
    os.mkdir(r'Data\Train\{}'.format((class_name)))

In [21]:
for class_name in all_annotationsdf['name'].unique():
    os.mkdir(r'Data\Test\{}'.format((class_name)))

In [22]:
for class_name in all_annotationsdf['name'].unique():
    os.mkdir(r'Data\Val\{}'.format((class_name)))

In [26]:
parent_path = 'Tomato Leaf Disease.v63i.voc\\train\\'

In [25]:
class_grouped = all_annotationsdf[all_annotationsdf['name'] == 'Mosaic Virus']
class_grouped

,filename,width,height,name,xmin,xmax,ymin,ymax
0,02cb8c4e-4542-4020-bb46-025de54d97ae___PSU_CG-...,512.0,512.0,Mosaic Virus,53.0,455.0,136.0,406.0
1,02cb8c4e-4542-4020-bb46-025de54d97ae___PSU_CG-...,512.0,512.0,Mosaic Virus,60.0,462.0,109.0,379.0
2,02cb8c4e-4542-4020-bb46-025de54d97ae___PSU_CG-...,512.0,512.0,Mosaic Virus,136.0,405.0,60.0,461.0
3,03b6acc7-41a6-4670-b47c-bef6d4baea18___PSU_CG-...,512.0,512.0,Mosaic Virus,143.0,403.0,44.0,386.0
4,03b6acc7-41a6-4670-b47c-bef6d4baea18___PSU_CG-...,512.0,512.0,Mosaic Virus,143.0,403.0,44.0,386.0
...,...,...,...,...,...,...,...,...
29757,ToMV-Tomate3_jpg.rf.38acd4414b3b495283a6c2e2d9...,512.0,512.0,Mosaic Virus,236.0,402.0,17.0,110.0
29758,ToMV-Tomate3_jpg.rf.a0e652b609ada47608e588ff73...,512.0,512.0,Mosaic Virus,153.0,287.0,325.0,497.0
29759,ToMV-Tomate3_jpg.rf.a0e652b609ada47608e588ff73...,512.0,512.0,Mosaic Virus,249.0,409.0,271.0,400.0
29760,ToMV-Tomate3_jpg.rf.a0e652b609ada47608e588ff73...,512.0,512.0,Mosaic Virus,261.0,437.0,131.0,236.0


In [28]:
class_grouped['name'].unique()

array(['Mosaic Virus'], dtype=object)

In [45]:
def Creat_Class(class_grouped, parent_path, dst, class_name):
    counter = 1
    for idx, row in tqdm(class_grouped.iterrows()):
        full_path_src = parent_path + row['filename']
        path_dst = '{}\\{}'.format(dst, class_name)
        image = Image.open(full_path_src)
        x_min, x_max, y_min, y_max = row['xmin'], row['xmax'], row['ymin'], row['ymax']
        cropped_image = image.crop((x_min, y_min, x_max, y_max))
        full_path = path_dst + '\\' + str(counter) + '.jpg'
        cropped_image.save('{}'.format(full_path))
        counter += 1    

In [46]:
for class_name in tqdm(class_diseas):
    class_grouped = all_annotationsdf[all_annotationsdf['name'] == class_name]
    parent_path = 'Tomato Leaf Disease.v63i.voc\\train\\'
    dst = 'Data\\Train'
    Creat_Class(class_grouped, parent_path, dst, class_name)


  0%|          | 0/9 [00:00<?, ?it/s]

4057it [01:03, 64.36it/s]
3141it [01:20, 39.23it/s]03<08:24, 63.08s/it]
4264it [02:02, 34.91it/s]23<08:31, 73.09s/it]
3888it [01:22, 47.18it/s]25<09:33, 95.50s/it]
5102it [01:25, 59.36it/s]47<07:31, 90.35s/it]
3327it [00:55, 60.48it/s]13<05:55, 88.78s/it]
4186it [01:11, 58.34it/s]08<03:51, 77.31s/it]
3165it [01:15, 41.80it/s]20<02:31, 75.50s/it]
2721it [00:55, 48.61it/s]36<01:15, 75.58s/it]
100%|██████████| 9/9 [11:32<00:00, 76.93s/it]


In [47]:
def ExtractInfo(path):
    # Extarct Images & Label Paths
    img_paths, lbl_paths = getPaths(path)
    
    # Extraxt Annotaions
    missing_counter = 0
    all_annotations = []
    for lbl_path in tqdm(lbl_paths):
        filepath = r'{}'.format(lbl_path)
        try :
            all_annotations.append(ExtractLabel(filepath))
        except FileNotFoundError:
            missing_counter += 1
    print('Number Of Missing Image {}'.format(missing_counter))


    all_annotations = [x for xs in tqdm(all_annotations) for x in xs]
    # Convert To DataFrame
    all_annotationsdf = pd.DataFrame(all_annotations, columns = ['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax'])

    # Convert Numerical Columns from object to int
    cols = ['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax']
    all_annotationsdf[cols] = all_annotationsdf[cols].astype(float)
    return all_annotationsdf

In [48]:
all_annotationsdf = ExtractInfo("Tomato Leaf Disease.v63i.voc\\valid\\")
all_annotationsdf

100%|██████████| 843/843 [00:03<00:00, 279.00it/s]


Number Of Missing Image 15


100%|██████████| 828/828 [00:00<00:00, 332811.09it/s]


,filename,width,height,name,xmin,xmax,ymin,ymax
0,0_jpg.rf.c894ea5ebaa9581203cc6cfb20558000.jpg,512.0,512.0,Septoria,262.0,422.0,194.0,361.0
1,0_jpg.rf.c894ea5ebaa9581203cc6cfb20558000.jpg,512.0,512.0,Septoria,74.0,250.0,194.0,361.0
2,11-40580_5_jpg.rf.290fb601cf36e565601e30589e47...,512.0,512.0,Yellow Leaf Curl Virus,139.0,213.0,1.0,134.0
3,11-40580_5_jpg.rf.290fb601cf36e565601e30589e47...,512.0,512.0,Yellow Leaf Curl Virus,1.0,125.0,105.0,324.0
4,11-40580_5_jpg.rf.290fb601cf36e565601e30589e47...,512.0,512.0,Yellow Leaf Curl Virus,142.0,190.0,360.0,470.0
...,...,...,...,...,...,...,...,...
2642,Worm-62_jpg.rf.2c7be2efb44fbe8fd371077c1bf97e1...,512.0,512.0,Leaf Miner,2.0,379.0,107.0,384.0
2643,Worm-6_jpg.rf.b74562b96b160c2e0cf3254664c9d19f...,512.0,512.0,Leaf Miner,54.0,396.0,21.0,513.0
2644,yellowing_JPG_jpg.rf.92ad8dfb877858ebc3122e471...,512.0,512.0,Yellow Leaf Curl Virus,24.0,248.0,302.0,401.0
2645,yellowing_JPG_jpg.rf.92ad8dfb877858ebc3122e471...,512.0,512.0,Yellow Leaf Curl Virus,263.0,510.0,283.0,408.0


In [54]:
for class_name in tqdm(class_diseas):
    class_grouped = all_annotationsdf[all_annotationsdf['name'] == class_name]
    parent_path = 'Tomato Leaf Disease.v63i.voc\\valid\\'
    dst = 'Data\\Val'
    Creat_Class(class_grouped, parent_path, dst, class_name)


319it [00:09, 34.94it/s]:00<?, ?it/s]
246it [00:05, 41.37it/s]:09<01:13,  9.16s/it]
323it [00:07, 43.21it/s]:15<00:50,  7.27s/it]
324it [00:05, 55.09it/s]:22<00:44,  7.37s/it]
315it [00:07, 41.66it/s]:28<00:33,  6.79s/it]
272it [00:06, 44.27it/s]:36<00:28,  7.08s/it]
338it [00:07, 47.92it/s]:42<00:20,  6.77s/it]
258it [00:05, 43.02it/s]:49<00:13,  6.87s/it]
252it [00:05, 42.67it/s]:55<00:06,  6.60s/it]
100%|██████████| 9/9 [01:01<00:00,  6.81s/it]


In [55]:
all_annotationsdf = ExtractInfo("Tomato Leaf Disease.v63i.voc\\test\\")
all_annotationsdf

100%|██████████| 165/165 [00:01<00:00, 163.66it/s]


Number Of Missing Image 1


100%|██████████| 164/164 [00:00<?, ?it/s]


,filename,width,height,name,xmin,xmax,ymin,ymax
0,039b47d574bc4bb8a14259a1cd96a741_jpg.rf.b8b089...,512.0,512.0,Early Blight,265.0,488.0,217.0,417.0
1,151b36defa8ccfe2fd90f4a342affd5a-tobacco-mosai...,512.0,512.0,Mosaic Virus,14.0,186.0,232.0,393.0
2,151b36defa8ccfe2fd90f4a342affd5a-tobacco-mosai...,512.0,512.0,Mosaic Virus,258.0,337.0,159.0,350.0
3,151b36defa8ccfe2fd90f4a342affd5a-tobacco-mosai...,512.0,512.0,Mosaic Virus,278.0,438.0,24.0,208.0
4,151b36defa8ccfe2fd90f4a342affd5a-tobacco-mosai...,512.0,512.0,Mosaic Virus,179.0,258.0,258.0,356.0
...,...,...,...,...,...,...,...,...
561,Worm-45_jpg.rf.72dcfcd3d24af9e53cabe4299d30feb...,512.0,512.0,Leaf Miner,1.0,152.0,251.0,382.0
562,Worm-53_jpg.rf.6f09680fc92985b9a7b2231a7c96b4e...,512.0,512.0,Leaf Miner,289.0,468.0,73.0,466.0
563,Worm-67_jpg.rf.fbe5022511601cbe4025f27f94b9bd4...,512.0,512.0,Leaf Miner,284.0,406.0,145.0,307.0
564,Worm-67_jpg.rf.fbe5022511601cbe4025f27f94b9bd4...,512.0,512.0,Leaf Miner,159.0,281.0,246.0,415.0


In [56]:
for class_name in tqdm(class_diseas):
    class_grouped = all_annotationsdf[all_annotationsdf['name'] == class_name]
    parent_path = 'Tomato Leaf Disease.v63i.voc\\test\\'
    dst = 'Data\\Test'
    Creat_Class(class_grouped, parent_path, dst, class_name)


77it [00:03, 23.76it/s]0:00<?, ?it/s]
71it [00:02, 27.83it/s]0:03<00:26,  3.26s/it]
72it [00:01, 65.35it/s]0:05<00:20,  2.86s/it]
41it [00:00, 43.59it/s]0:06<00:12,  2.08s/it]
50it [00:00, 57.05it/s]0:07<00:08,  1.65s/it]
68it [00:01, 45.83it/s]0:08<00:05,  1.38s/it]
87it [00:01, 45.86it/s]0:10<00:04,  1.42s/it]
56it [00:01, 40.51it/s]0:12<00:03,  1.58s/it]
44it [00:00, 46.60it/s]0:13<00:01,  1.52s/it]
100%|██████████| 9/9 [00:14<00:00,  1.63s/it]
